In [12]:
import pandas as pd
import numpy as np
import json

In [13]:
graph = "memoryalpha"
embeddings = "dogtag_bgem3"
communities = "leiden"

In [14]:
embeddings_file = "./_input/node_embeddings/" + embeddings + "/" + graph + ".json"
url_embeddings_file = "./_input/url_embeddings/" + graph + ".json"
communities_file = "./_input/communities/" + communities + "/" + graph + ".txt"
community_embeddings_file = "./_input/community_embeddings/" + communities + "_" + embeddings + "/" + graph + ".json"

In [15]:
df = pd.read_json(embeddings_file)
df_T = df.T

In [16]:
# df_T

In [17]:
# df_T.loc[0]

In [18]:
df_url = pd.read_json(url_embeddings_file)
df_url_T = df_url.T

In [19]:
# df_url_T

In [20]:
# df_url_T.loc[2239]

In [21]:
communities = []
with open(communities_file) as cbf:
    for line in cbf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        communities.append(numbers_set)

In [22]:
community_embeddings = []

for community in communities:

    embeddings = []

    for node in community:
        try:
            selected_embedding = df_T.loc[node]
            embeddings.append(selected_embedding)
        except KeyError:
            try:
                selected_embedding = df_url_T.loc[node]
                embeddings.append(selected_embedding)
            except KeyError:
                print(str(node) + " --- Key not found anywhere!")

    community_embedding = np.average(embeddings, axis=0)
    community_embeddings.append(community_embedding)

embeddings_dict = {
    str(i): {str(j): val for j, val in enumerate(embedding)}
    for i, embedding in enumerate(community_embeddings)
}

with open(community_embeddings_file, "w") as json_file:
    json.dump(embeddings_dict, json_file, indent=4)

373547 --- Key not found anywhere!
64312 --- Key not found anywhere!
163342 --- Key not found anywhere!
391157 --- Key not found anywhere!
